In [1]:
import rasterio as rio

In [2]:
granule_loc = 'D:/canopy_data/error_fixing/granule.tif'

src = rio.open(granule_loc)

In [3]:
src.meta

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': None,
 'width': 3340,
 'height': 3340,
 'count': 17,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(8.983152841195215e-05, 0.0, 19.58929190620917,
        0.0, -8.983152841195215e-05, -0.40954193803008987)}

In [4]:
src.profile

{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 3340, 'height': 3340, 'count': 17, 'crs': CRS.from_epsg(4326), 'transform': Affine(8.983152841195215e-05, 0.0, 19.58929190620917,
       0.0, -8.983152841195215e-05, -0.40954193803008987), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}

In [1]:
from convert_rasters import convert_granule

ModuleNotFoundError: No module named 'rasterio'

In [ ]:
granule_path = 'enter_here'

dest_dir = 's3://canopy-production-ml/full_congo_basin/test_conversion/'

result = convert_granule(granule_path, dest_dir)

result